In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os



In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
DATA_DIR = '/home/chicm/ml/kgdata/species'

DATA_DIR = '/home/chicm/data/cervc/clean640'
TRAIN_DIR = DATA_DIR+'/train'
TEST_DIR = DATA_DIR + '/test'
VAL_DIR = DATA_DIR + '/valid'
RESULT_DIR = DATA_DIR + '/results3'

batch_size = 64


In [5]:
os.chdir(TRAIN_DIR)
train_filenames = glob('*/*.jpg')

os.chdir(VAL_DIR)
val_filenames = glob('*/*.jpg')

#os.chdir(TEST_DIR)
#test_filenames = glob('*/*.jpg')

print(train_filenames[:5])
print(val_filenames[:5])
#print(test_filenames[:5])

['Type_1/5770.jpg', 'Type_1/6360.jpg', 'Type_1/3811.jpg', 'Type_1/6096.jpg', 'Type_1/1580.jpg']
['Type_1/560.jpg', 'Type_1/3049.jpg', 'Type_1/2749.jpg', 'Type_1/6420.jpg', 'Type_1/5983.jpg']


In [6]:
img_path = TRAIN_DIR+'/'

In [7]:
def one_hot(cls):
    if cls == 'Type_1':
        return [1, 0, 0]
    elif cls == 'Type_2':
        return [0, 1, 0]
    elif cls == 'Type_3':
        return [0, 0, 1]
    else:
        return None

In [8]:
y_train = [item.split('/')[0] for item in train_filenames]
y_train = [one_hot(item) for item in y_train]
y_train = np.array(y_train, np.float32)
 
print(y_train.shape)
print(y_train[:3])

(7021, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]


In [9]:
y_test = [item.split('/')[0] for item in val_filenames]
y_test = [one_hot(item) for item in y_test]
y_test = np.array(y_test, np.float32)
 
print(y_test.shape)
print(y_test[:3])

(598, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]


In [11]:
#image reseize & centering & crop 

def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized


x_train = []
for i, file_path in enumerate(train_filenames):
    #read image
    img = cv2.imread(TRAIN_DIR+'/'+file_path)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    x_train.append(img)

x_train = np.array(x_train)
print(x_train.shape)


(7021, 224, 224, 3)


In [12]:
x_test = []
for i, file_path in enumerate(val_filenames):
    #read image
    img = cv2.imread(VAL_DIR+'/'+file_path)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    x_test.append(img)

x_test = np.array(x_test)
print(x_test.shape)

(598, 224, 224, 3)


In [13]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")
img_path = DATA_DIR+"/test/unknown/"

test_names = []
file_paths = []

for i in range(len(sample_submission)):
    test_names.append(sample_submission.ix[i][0])
    file_paths.append( img_path + str(sample_submission.ix[i][0]))
    
test_names = np.array(test_names)
print(test_names.shape)
print(test_names[:5])
print(file_paths[:5])

(512,)
['0.jpg' '1.jpg' '2.jpg' '3.jpg' '4.jpg']
['/home/chicm/data/cervc/clean640/test/unknown/0.jpg', '/home/chicm/data/cervc/clean640/test/unknown/1.jpg', '/home/chicm/data/cervc/clean640/test/unknown/2.jpg', '/home/chicm/data/cervc/clean640/test/unknown/3.jpg', '/home/chicm/data/cervc/clean640/test/unknown/4.jpg']


In [14]:
test_images = []
for file_path in file_paths:
    #read image
    img = cv2.imread(file_path)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    test_images.append(img)
    
    path, ext = os.path.splitext( os.path.basename(file_paths[0]) )

test_images = np.array(test_images)

## split train and val

In [15]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [16]:
print(y_test[:1])
print(y_train[:1])

[[ 1.  0.  0.]]
[[ 1.  0.  0.]]


In [38]:
print(val_filenames[202])
print(y_test[202])
print(train_filenames[2201])
print(y_train[2201])

for i in range(500):
    print(y_test[i], one_hot(val_filenames[i].split('/')[0]))


Type_3/954.jpg
[ 0.  0.  1.]
Type_3/2295.jpg
[ 0.  0.  1.]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1.  0.  0.] [1, 0, 0]
[ 1. 

## Use VGG

In [17]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import SGD, RMSprop, Adam

img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

Using TensorFlow backend.


In [40]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(3, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
for layer in model.layers[:25]:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.00002, momentum=0.9),
              metrics=['accuracy'])
#model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [41]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

batch_size = 64
epochs = 50
w_file = RESULT_DIR + '/vgg16_1.h5'

#model.fit(x_train, y_train, epochs = epochs,
#         verbose = 2, validation_data=(x_test, y_test),
#         callbacks=[ModelCheckpoint(w_file, monitor='val_acc', save_best_only=True)])

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        zoom_range=0.2,
        channel_shift_range=20,
        width_shift_range=0.1,
        height_shift_range=0.05, 
        shear_range=0.1,
        vertical_flip=True,
        horizontal_flip=True)
train_datagen.fit(x_train)

history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test), verbose=2,
    callbacks=[ModelCheckpoint(w_file, monitor='val_acc', save_best_only=True)]
)

Epoch 1/50
123s - loss: 1.2888 - acc: 0.3516 - val_loss: 1.1919 - val_acc: 0.2659
Epoch 2/50


KeyboardInterrupt: 

In [17]:
test_images = test_images.astype('float32')
test_images /= 255

In [18]:
model.load_weights('VGG16-transferlearning.model')
predictions = model.predict(test_images)

In [19]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")

for i, name in enumerate(test_names):
    sample_submission.loc[sample_submission['name'] == name, 'invasive'] = predictions[i]

sample_submission.to_csv("submit1.csv", index=False)